Hint :- Please dont forget to start "Complaints & Review Graph" in Neo4J

In [77]:
############################ Importing All Libreries & collecting input ################################
from py2neo import Graph,Node,Relationship
import pandas as pd
import ipywidgets as ipw
df = pd.read_excel("IDs.xlsx")
g = Graph()
i = ipw.Dropdown(
    options=list(df["Mobile Names_Neo4J"].dropna()),
    description = "Select Mobile - ", )
i


Dropdown(description='Select Mobile - ', options=('NONE', 'Samsung C3500 Monte Bar', 'Samsung Intercept', 'Sam…

In [78]:
################################## Verifing Selected Input ######################################
input = i.value
input

'Samsung Galaxy S6 edge+ Duos'

In [1]:
########################################## Neo4J Query ###########################################
q = '''MATCH p=(r:Rating)<-[:RATING_DETAILS]-(rev:Review)-[d:REVIEW_ON]->(m:Mobile)-[:IS_A]->(t:Mobile_Type)
WITH "''' + input + '''" as mob 
WHERE (r.Rating = "10" OR r.Rating = "9" OR r.Rating = "8" OR r.Rating = "7" OR r.Rating = "6") AND m.Mobile_Name = mob
WITH COUNT(*) AS num
WITH num , apoc.number.exact.div(toString(num),'828.15',3,'CEILING') AS ind
RETURN {Postive_Ratings : num , index : ind } AS result

UNION ALL

MATCH p=(r:Rating)<-[:RATING_DETAILS]-(rev:Review)-[d:REVIEW_ON]->(m:Mobile)-[:IS_A]->(t:Mobile_Type)
WITH "''' + input + '''" as mob 
WHERE (r.Rating = "5" OR r.Rating = "4" OR r.Rating = "3" OR r.Rating = "2" OR r.Rating = "1") AND m.Mobile_Name = mob
WITH COUNT(*) AS num
WITH num , apoc.number.exact.div(toString(num),'828.15',3,'CEILING') AS ind
RETURN {Negative_Ratings : num , index : ind } AS result

UNION ALL
MATCH p=(s:Sentiment) <-[:SENTIMENT_DETAILS]-(c:Complaint) -[d:COMPLAINT_AGAINST]-> 
(o:Order)-[:PURCHASED_DETAILS]->(m:Mobile)-[:IS_A]->(t:Mobile_Type)
WITH "''' + input + '''" as mob 
WHERE s.Sentiment = "Positive" AND m.Mobile_Name = mob
WITH COUNT(*) AS num
WITH num , apoc.number.exact.div(toString(num),'80',3,'CEILING') AS ind
RETURN {Positive_Sentiment_Complaints : num , index : ind } AS result

UNION ALL
MATCH p=(s:Sentiment) <-[:SENTIMENT_DETAILS]-(c:Complaint) -[d:COMPLAINT_AGAINST]->
(o:Order)-[:PURCHASED_DETAILS]->(m:Mobile)-[:IS_A]->(t:Mobile_Type)
WITH "''' + input + '''" as mob  
WHERE s.Sentiment = "Negative" AND m.Mobile_Name = mob
WITH COUNT(*) AS num
WITH num , apoc.number.exact.div(toString(num),'80',3,'CEILING') AS ind
RETURN {Negative_Sentiments_Complaints : num , index : ind } AS result'''

TypeError: can only concatenate str (not "method") to str

In [80]:
############################### Passing query to Neo4J & getting results ###############################
df = g.run(q).to_data_frame()
print('"' + input , '" has the following results')
df['result'].values

"Samsung Galaxy S6 edge+ Duos " has the following results


array([{'Postive_Ratings': 1093, 'index': '1.32'},
       {'index': '1.65', 'Negative_Ratings': 1366},
       {'Positive_Sentiment_Complaints': 107, 'index': '1.34'},
       {'Negative_Sentiments_Complaints': 112, 'index': '1.4'}],
      dtype=object)